In [ ]:
import os
import time
import hashlib
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# This line define the path to the "malicious.pdf" file which is in the Colab environment
MONITORED_FILE = r"/content/malicious.pdf"

# The hash function meant for file integrity check
def get_file_hash(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        # This part reads and updates the hash string value in blocks of 4K
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

# This dictionary stores original file hash used for comparison
file_hashes = {}

# Initialize file hash
def init_file_hashes():
    print(f"Initializing file hashes for {MONITORED_FILE}...")
    file_hashes[MONITORED_FILE] = get_file_hash(MONITORED_FILE)

# This function defines the rule for detecting ransomware activity
def detect_ransomware(file_path, event_type):
    print(f"Event detected: {event_type} on {file_path}")

    # This condition checks if the file is renamed with '.enc' extension (It is common in ransomware attacks)
    if event_type == 'modified' and file_path.endswith('.enc'):
        print(f"Suspicious activity detected: File renamed to {file_path} (possible ransomware encryption)")
        trigger_mitigation(file_path)

    # This condition checks if the file was modified unexpectedly or not
    if event_type == 'modified' and file_path == MONITORED_FILE:
        print(f"File modification detected: {file_path}")
        # This part compares the current file hash with the original one in order to detect unauthorized changes
        current_hash = get_file_hash(file_path)
        if current_hash != file_hashes.get(file_path):
            print(f"File content has been modified unexpectedly: {file_path}")
            trigger_mitigation(file_path)

def trigger_mitigation(file_path):
    # A sample mitigation: it sends alert and block the file (additional mitigation can be added)
    print(f"Mitigation triggered for {file_path}: Sending alert and blocking file access")
    # It send alert (Which can be an email or message to sysadmin)
    send_alert(file_path)
    # Optionally block file (example for Linux-based systems in Colab)
    os.chmod(file_path, 0o000)  # Change file permissions to prevent further access

def send_alert(file_path):
    # A sample alert function, which sends a simple notification to the system administrator
    print(f"ALERT: Potential ransomware activity detected on {file_path}. Please check the system immediately.")

# This class creates a custom event handler for monitoring the file
class RansomwareDetectorHandler(FileSystemEventHandler):
    def on_modified(self, event):
        print(f"File modified: {event.src_path}")
        if event.is_directory:
            return
        detect_ransomware(event.src_path, 'modified')

    def on_created(self, event):
        print(f"File created: {event.src_path}")
        if event.is_directory:
            return
        detect_ransomware(event.src_path, 'created')

    def on_deleted(self, event):
        print(f"File deleted: {event.src_path}")
        if event.is_directory:
            return
        print(f"File deletion detected: {event.src_path}")

# This is the main function used to set up monitoring
def monitor_files():
    init_file_hashes()  # This initializes the hash table with the current file hash

    event_handler = RansomwareDetectorHandler()
    observer = Observer()

    # This snippet monitors the specific file for changes (It monitors directory only)
    observer.schedule(event_handler, os.path.dirname(MONITORED_FILE), recursive=False)
    observer.start()
    print(f"Monitoring started for {MONITORED_FILE}...")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

if __name__ == "__main__":
    monitor_files()


Initializing file hashes for /content/malicious.pdf...
Monitoring started for /content/malicious.pdf...
File modified: /content
File modified: /content


In [8]:
!pip install watchdog


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.9 MB/s eta 0:00:00
